In [53]:
# 模型的优化训练是怎么做的
# 数据的清洗 和 选择
# 增加数据特征

In [5]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split

In [6]:
train = pd.read_csv('./data/train_feature.csv')
train.head()

,x_mean,x_std,x_quantile,x_median,x_kurt,x_skew,y_mean,y_std,y_quantile,y_median,...,tuo_dist_kurt,tuo_dist_skew,wei_dist_mean,wei_dist_std,wei_dist_quantile,wei_dist_median,wei_dist_kurt,wei_dist_skew,s,type
0,6.247895e+06,4438.069089,6.246627e+06,6.246627e+06,13.235139,3.712603,5.239679e+06,5939.181795,5.241151e+06,5.241260e+06,...,15.639965,-3.917515,0.186612,0.040645,0.175043,0.175043,11.122571,3.532651,8.723267e+08,拖网
1,6.173433e+06,12250.695539,6.165485e+06,6.165586e+06,-0.359746,1.152952,5.197881e+06,6039.038082,5.187879e+06,5.201786e+06,...,-0.535744,-1.086920,0.104653,0.070213,0.058759,0.059132,-0.532114,1.089043,7.098153e+08,刺网
2,7.053152e+06,12171.034972,7.028056e+06,7.057604e+06,0.108375,-1.241603,6.089065e+06,19760.522701,6.061634e+06,6.089506e+06,...,-0.086825,-0.682148,0.690907,0.094144,0.566603,0.739652,-0.163807,-0.662110,3.587348e+09,围网
3,6.711096e+06,60764.037193,6.658979e+06,6.663841e+06,-0.927983,0.631317,5.459586e+06,25339.521511,5.433949e+06,5.453345e+06,...,0.931561,1.115817,0.874476,0.142671,0.754016,0.804881,0.763413,1.095577,1.604409e+10,围网
4,6.194958e+06,12947.725471,6.178915e+06,6.199598e+06,-0.727048,-0.612727,5.134912e+06,24345.461517,5.113110e+06,5.128339e+06,...,-0.994802,-0.638356,0.461491,0.134213,0.162758,0.528028,0.299261,-1.231053,4.470275e+09,拖网


In [7]:
label_dict = {'拖网':0, '刺网':1, '围网':2}
train['type'] = train['type'].map(label_dict)
label = np.array(train['type'])
label

array([0, 1, 2, ..., 1, 2, 0])

In [8]:
x = np.array(train.drop('type', axis=1))
x.shape

(7000, 37)

In [9]:
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score

In [47]:
param = {'num_leaves': 31,     #由于lightGBM是leaves_wise生长，官方说法是要小于2^max_depth
         'objective':'multiclassova',
         'num_class':3,
         'learning_rate': 0.01,
         "boosting": "gbdt",
#          "feature_fraction": 0.9,
#          "bagging_fraction": 0.9,
         "lambda_l1": 0.1,
         "verbosity": -1,
        }

In [62]:
folds = KFold(n_splits=5, shuffle=True, random_state=42)
model_list = []
val_f1score = []
for fold, (train_idx, val_idx) in enumerate(folds.split(x, label)):
    print('fold = ', fold+1)
   
    curr_train_data = lgb.Dataset(x[train_idx], label[train_idx])
    curr_val_data = lgb.Dataset(x[val_idx], label[val_idx])
    
    num_boost_round = 1000
    booster_model = lgb.train(param, 
                                curr_train_data, 
                                num_boost_round, 
                                valid_sets = [curr_train_data, curr_val_data], 
                                verbose_eval=500,
                                early_stopping_rounds = 1000)
   
    label_pred = booster_model.predict(x[val_idx], num_iteration=booster_model.best_iteration)
    label_pred_encode = np.argmax(label_pred, axis=1)
    curr_f1score = f1_score(label_pred_encode, label[val_idx], average='macro')
    print(fold + 1, '_validation f1_score is ', curr_f1score)
    model_list.append(booster_model)
    val_f1score.append(curr_f1score)

fold =  1
Training until validation scores don't improve for 1000 rounds
[500]	training's multi_logloss: 0.149484	valid_1's multi_logloss: 0.271465
[1000]	training's multi_logloss: 0.0712172	valid_1's multi_logloss: 0.25271
Did not meet early stopping. Best iteration is:
[1000]	training's multi_logloss: 0.0712172	valid_1's multi_logloss: 0.25271
1 _validation f1_score is  0.8814939900333293
fold =  2
Training until validation scores don't improve for 1000 rounds
[500]	training's multi_logloss: 0.150985	valid_1's multi_logloss: 0.28273
[1000]	training's multi_logloss: 0.0725656	valid_1's multi_logloss: 0.267597
Did not meet early stopping. Best iteration is:
[1000]	training's multi_logloss: 0.0725656	valid_1's multi_logloss: 0.267597
2 _validation f1_score is  0.8715722668275584
fold =  3
Training until validation scores don't improve for 1000 rounds
[500]	training's multi_logloss: 0.151202	valid_1's multi_logloss: 0.276101
[1000]	training's multi_logloss: 0.0728592	valid_1's multi_logl

In [63]:
print(val_f1score)

[0.8814939900333293, 0.8715722668275584, 0.8711870884521332, 0.8794480784807198, 0.8832410259592066]


In [64]:
sorted_idx = np.array(val_f1score).argsort()
sorted_idx

array([2, 1, 3, 0, 4])

In [65]:
res_list = []
for m in model_list:
    test_pred = m.predict(test, num_iteration = m.best_iteration)
    test_pred_encode = np.argmax(test_pred, axis=1)
    res_list.append(test_pred_encode)
res_list

[array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0])]

In [71]:
temp = np.array(res_list).T
temp

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

In [93]:
np.argmax(np.bincount(temp[1]))

0

In [82]:
res_final = []
for i in range(temp.shape[0]):
    res_final.append(np.argmax(np.bincount(temp[i])))
res_final

[0,
 0,
 0,
 1,
 2,
 1,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 1,
 0,
 2,
 1,
 0,
 0,
 2,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 2,
 0,
 0,
 2,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 2,
 2,
 0,
 1,
 1,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 2,
 0,
 2,
 0,
 1,
 2,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 2,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 2,
 1,
 0,
 2,
 1,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 2,
 0,
 0,
 2,
 0,
 1,
 2,
 0,
 0,
 2,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 2,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 2,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 1,
 2,
 0,
 0,
 2,
 1,
 0,
 2,
 0,
 0,
 1,
 2,
 0,
 0,
 2,
 0,
 1,
 2,
 2,
 2,
 0,
 0,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 0,
 1,
 2,
 0,
 0,
 2,
 0,
 2,
 1,
 0,
 0,
 2,


In [87]:
np.array(res_final).sum()

1192

In [83]:
res = pd.DataFrame( {'id':test_fea['id'], 'type':res_final})
num2label = {0: '拖网', 1 :'刺网', 2: '围网'}
res['type'] = res['type'].map(num2label)
res

,id,type
0,8031,拖网
1,8325,拖网
2,7660,拖网
3,8408,刺网
4,7743,围网
...,...,...
1995,8141,拖网
1996,8121,拖网
1997,8575,拖网
1998,8717,拖网


In [90]:
res.to_csv('./res_002.csv', header=False, index=False)
print('done!')

done!


In [84]:
num = len(res[res['type']=='围网'])
print(num, num/ 2000)

468 0.234


In [85]:
num = len(res[res['type']=='刺网'])
print(num, num/ 2000)

256 0.128


In [86]:
num = len(res[res['type']=='拖网'])
print(num, num/ 2000)

1276 0.638


In [57]:
test_fea = pd.read_csv('./data/test_feature.csv')
test_fea
test = np.array(test_fea.drop('id', axis=1))

In [31]:
test_pred = booster_model.predict(test, num_iteration=booster_model.best_iteration)
test_pred_encode = np.argmax(test_pred, axis=1)
len(test_pred_encode)

2000

In [32]:
test_pred_encode

array([0, 0, 0, ..., 0, 0, 0])

In [33]:
test_pred_encode.sum()

1202

In [40]:
res = pd.DataFrame( {'id':test_fea['id'], 'type':test_pred_encode})
num2label = {0: '拖网', 1 :'刺网', 2: '围网'}
res['type'] = res['type'].map(num2label)
res

,id,type
0,8031,拖网
1,8325,拖网
2,7660,拖网
3,8408,刺网
4,7743,围网
...,...,...
1995,8141,拖网
1996,8121,拖网
1997,8575,拖网
1998,8717,拖网


In [36]:
res.to_csv('./data/res_001.csv', header=False, index=False)
print('done!')

done!


In [37]:
num = len(res[res['type']=='围网'])
print(num, num/ 2000)

477 0.2385


In [38]:
num = len(res[res['type'] == '刺网'])
print(num, num/ 2000)

248 0.124


In [39]:
num = len(res[res['type']=='拖网'])
print(num, num/2000)

1275 0.6375
